In [185]:
import pandas as pd
import numpy as np
import html
import matplotlib as plt
import re
pd.set_option('display.max_columns', None)

In [186]:
ahistoric_df = pd.read_csv("./data_csvs/Open_Restaurant_Applications__Historic__20231202.csv")

In [187]:
ahistoric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14428 entries, 0 to 14427
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   objectid                                  14428 non-null  int64  
 1   globalid                                  14428 non-null  object 
 2   Seating Interest (Sidewalk/Roadway/Both)  14428 non-null  object 
 3   Restaurant Name                           14427 non-null  object 
 4   Legal Business Name                       14427 non-null  object 
 5   Doing Business As (DBA)                   14416 non-null  object 
 6   Building Number                           12776 non-null  object 
 7   Street                                    14428 non-null  object 
 8   Borough                                   14428 non-null  object 
 9   Postcode                                  14428 non-null  int64  
 10  Business Address                  

In [188]:
ahistoric_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,{FD87ABAA-860E-4762-845D-8F0403D0246B},roadway,MADAME BONTE,BEANHOUSE LLC,MADAME BONTE,NaN,318 EAST 84 STREET,Manhattan,10028,"318 EAST 84 STREET, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,{3B07E4C0-07B7-4079-8333-64446CC3EE03},sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,{137C575D-DC14-4F9D-83D9-A3FFE513B3B8},sidewalk,AMERICAS CAFE &amp; GRILL,68TH GRILL INC,AMERICAS CAFE &amp; GRILL,NaN,1159 3 AVENUE,Manhattan,10065,"1159 3 AVENUE, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,{15270732-2A78-4C24-89DD-BE8DD916F115},roadway,SUSHI SEKI,SEKI INC,SUSHI SEKI,undefined,208 WEST 23 STREET,Manhattan,10011,"208 WEST 23 STREET, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,Aug-82,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,{EF9C8173-91D1-496E-8BD4-B02BEADC2A21},roadway,Sami &amp; Susu,AMIR NATHAN,Sami & Susu,NaN,190 ORCHARD STREET,Manhattan,10002,"190 ORCHARD STREET, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,Jul-61,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,{108C6784-54DB-480B-8627-CF0495D27A0E},both,KING OF SPADES INC.,KING OF SPADES INC.,KING OF SPADES INC.,1425,COLLEGE POINT BLVD,Queens,11356,"1425 COLLEGE POINT BLVD, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,Feb-05,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,{7B8EC275-D470-4106-819E-03FC0E1F9717},both,LA QUEEN TEA HOUSE INC,LA QUEEN TEA HOUSE INC,LA QUEEN TEA HOUSE INC,752B,61ST ST,Brooklyn,11220,"752B 61ST ST, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,{476f88d2-d3e8-4e83-8686-8bac2f752cf5},sidewalk,aahar indina cuisine,vidhan bhatt inc.,vidhan bhatt inc.,10,murray street,Manhattan,10007,"10 murray street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,ff0d04ef-1d9a-47d6-8f20-8bdbdbce2792,sidewalk,le cafe coffee,le cafe coffee llc,le cafe coffee llc,145,fourth av,Manhattan,10003,"145 fourth av, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


# Cleaning Start

- Step 1: Remove Duplicates, and columns which don't have any values
- Step 2: Remove {} brackets from globalid
- Step 3: Capitalize Seating Interest Column
- Step 4: Remove HTML Encoding from the Columns
- Step 5: Capitalize Name, Legal Business Name, Doing Business As, and Street Columns, and standardize suffixes and abbreviations
- Step 6: Remove Building Number from Street Addresses
- Step 7: Convert numbers into Ordinal Numbers
- Step 8: Recreate the Business Address Column with the cleaned fields
- Step 9: Convert corrupted SLA Licence numbers to a default value (0000000)
- Step 10: Save as cleaned CSV

In [189]:
#1.Remove Duplicates
ahistoric_df=ahistoric_df.drop_duplicates()

#2.Standardize Global ID
ahistoric_df['globalid']=ahistoric_df['globalid'].str.upper()
ahistoric_df['globalid']=ahistoric_df['globalid'].str.replace('{','')
ahistoric_df['globalid']=ahistoric_df['globalid'].str.replace('}','')

#3.Capitalize Seating Interest
ahistoric_df['Seating Interest (Sidewalk/Roadway/Both)']=ahistoric_df['Seating Interest (Sidewalk/Roadway/Both)'].str.capitalize()

#4.Remove any HTML Encoding from the columns.
def decode_html_entities(column):
    return column.apply(lambda x: html.unescape(str(x)))

ahistoric_df['Restaurant Name'] = decode_html_entities(ahistoric_df['Restaurant Name'])
ahistoric_df['Doing Business As (DBA)'] = decode_html_entities(ahistoric_df['Doing Business As (DBA)'])
ahistoric_df['Legal Business Name'] = decode_html_entities(ahistoric_df['Legal Business Name'])
ahistoric_df

C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\1552543227.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ahistoric_df['globalid']=ahistoric_df['globalid'].str.replace('{','')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\1552543227.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ahistoric_df['globalid']=ahistoric_df['globalid'].str.replace('}','')


,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,MADAME BONTE,BEANHOUSE LLC,MADAME BONTE,NaN,318 EAST 84 STREET,Manhattan,10028,"318 EAST 84 STREET, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,AMERICAS CAFE & GRILL,68TH GRILL INC,AMERICAS CAFE & GRILL,NaN,1159 3 AVENUE,Manhattan,10065,"1159 3 AVENUE, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,SUSHI SEKI,SEKI INC,SUSHI SEKI,undefined,208 WEST 23 STREET,Manhattan,10011,"208 WEST 23 STREET, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,Aug-82,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,AMIR NATHAN,Sami & Susu,NaN,190 ORCHARD STREET,Manhattan,10002,"190 ORCHARD STREET, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,Jul-61,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,KING OF SPADES INC.,KING OF SPADES INC.,KING OF SPADES INC.,1425,COLLEGE POINT BLVD,Queens,11356,"1425 COLLEGE POINT BLVD, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,Feb-05,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,LA QUEEN TEA HOUSE INC,LA QUEEN TEA HOUSE INC,LA QUEEN TEA HOUSE INC,752B,61ST ST,Brooklyn,11220,"752B 61ST ST, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,aahar indina cuisine,vidhan bhatt inc.,vidhan bhatt inc.,10,murray street,Manhattan,10007,"10 murray street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,FF0D04EF-1D9A-47D6-8F20-8BDBDBCE2792,Sidewalk,le cafe coffee,le cafe coffee llc,le cafe coffee llc,145,fourth av,Manhattan,10003,"145 fourth av, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [190]:
#5. Function to Capitalize the columns and handle resulting inconsistencies.
def capitalize_column(dataframe, column):
    dataframe[column]=dataframe[column].str.title()
    
    #Standardize Business Suffixes
    pattern_1 = r'\bInc\b(?!\.)'
    pattern_2 = r'\bLlc\b(?!\.)'
    pattern_3 = r'\bCorp\b(?!\.)'
    pattern_4 = r'\bIncorporated\b(?!\.)'
    dataframe[column] = dataframe[column].str.replace(pattern_1,'Inc.')
    dataframe[column] = dataframe[column].str.replace(pattern_2,'LLC')
    dataframe[column] = dataframe[column].str.replace(pattern_3,'Corp.')
    dataframe[column] = dataframe[column].str.replace(pattern_4,'Inc.')
    
    #Standardize Location Abbreviations
    pattern_st = r'\bSt$|\bSt\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_st,'Street')
    pattern_av = r'\bAv$|\bAv\.$|\bAve$|\bAve\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_av,'Avenue')
    pattern_ln = r'\bLn$|\bLn\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_ln,'Lane')
    pattern_ln = r'\bBlvd$|\bBlvd\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_ln,'Boulevard')
    
    #Change words ending in "'S" to ending in "'s" or "'T" to ending in "'t"
    pattern = r"(\w+)'S\b"
    replacement = lambda match: match.group(1).capitalize() + "'s"
    dataframe[column] = dataframe[column].str.replace(pattern, replacement, regex=True)
    pattern = r"(\w+)'T\b"
    replacement = lambda match: match.group(1).capitalize() + "'t"
    dataframe[column] = dataframe[column].str.replace(pattern, replacement, regex=True)
    return dataframe

ahistoric_df = capitalize_column(ahistoric_df,'Restaurant Name')
ahistoric_df = capitalize_column(ahistoric_df,'Doing Business As (DBA)')
ahistoric_df = capitalize_column(ahistoric_df,'Street')
ahistoric_df = capitalize_column(ahistoric_df,'Legal Business Name')

C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\72483577.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_1,'Inc.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\72483577.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_2,'LLC')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\72483577.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_3,'Corp.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\72483577.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_4,'Inc.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_31428\72483577.py:17: Futu

In [191]:
#6. aking out Building Number from Street Column
def extract_building_number_and_remove(address):
    words = address.split()
    for word in words:
        if word.isdigit():
            address = address.replace(word, '').strip()
            return pd.Series([word, address], index=['Building Number', 'Street'])
    return pd.Series([None, address], index=['Building Number', 'Street'])    # If no number is found, return None
    return None, address
mask = ahistoric_df['Building Number'].isna() | (ahistoric_df['Building Number'] == 'Undefined') | (ahistoric_df['Building Number'] == 'undefined')
ahistoric_df.loc[mask, ['Building Number', 'Street']] = ahistoric_df.loc[mask, 'Street'].apply(extract_building_number_and_remove)

ahistoric_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84 Street,Manhattan,10028,"318 EAST 84 STREET, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68Th Grill Inc.,Americas Cafe & Grill,1159,3 Avenue,Manhattan,10065,"1159 3 AVENUE, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23 Street,Manhattan,10011,"208 WEST 23 STREET, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,Aug-82,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 ORCHARD STREET, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,Jul-61,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 COLLEGE POINT BLVD, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,Feb-05,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61St Street,Brooklyn,11220,"752B 61ST ST, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 murray street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,FF0D04EF-1D9A-47D6-8F20-8BDBDBCE2792,Sidewalk,Le Cafe Coffee,Le Cafe Coffee LLC,Le Cafe Coffee LLC,145,Fourth Avenue,Manhattan,10003,"145 fourth av, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [192]:
#7. Make ordinal numbers from standalone numbers
def make_ordinal(number_str):
    # Use regex to find standalone numbers
    matches = re.findall(r'\b\d+\b', str(number_str))
    
    # Apply ordinal conversion to standalone numbers
    for match in matches:
        number = int(match)
        if number % 100 in [11, 12, 13]:
            suffix = 'th'
        else:
            suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(number % 10, 'th')
        number_with_suffix = str(number) + suffix
        
        # Replace the standalone number in the original string
        number_str = re.sub(r'\b{}\b'.format(match), number_with_suffix, str(number_str))

    return number_str

# Make Ordinal Numbers from Words
def words_to_ordinal(address):
    word_to_number = {'First': '1st', 'Second': '2nd', 'Third': '3rd', 'Fourth': '4th', 'Fifth': '5th', 'Sixth': '6th' \
                     , 'Seventh': '7th', 'Eight': '8th', 'Ninth': '9th', 'Tenth': '10th'}
    
    # Use regex to find words in the address and replace them with ordinal numbers
    for word, ordinal in word_to_number.items():
        address = re.sub(r'\b{}\b'.format(word), ordinal, address)

    return address

# Convert Capital suffixes to small
def convert_suffixes(number_str):
    # Use regex to find words with numbers followed by 'Th', 'Rd', 'Nd', 'St'
    matches = re.finditer(r'(\d+)([TtRrNnSs][HhDdNnTt])\b', str(number_str))
    
    # Apply the conversion to matched words
    for match in matches:
        number, suffix = match.groups()
        number_str = number_str.replace(match.group(0), f"{number}{suffix.lower()}")
    
    return number_str

ahistoric_df['Street']=ahistoric_df['Street'].apply(convert_suffixes)
ahistoric_df['Street']=ahistoric_df['Street'].apply(make_ordinal)
ahistoric_df['Street']=ahistoric_df['Street'].apply(words_to_ordinal)
ahistoric_df['Legal Business Name']=ahistoric_df['Legal Business Name'].apply(convert_suffixes)
ahistoric_df['Restaurant Name']=ahistoric_df['Restaurant Name'].apply(convert_suffixes)
ahistoric_df['Doing Business As (DBA)']=ahistoric_df['Doing Business As (DBA)'].apply(convert_suffixes)

In [193]:
# 8. Fix the Business Address column
ahistoric_df['Business Address'] = ahistoric_df['Building Number'] + " "+ ahistoric_df['Street'] + ", " +ahistoric_df['Borough'] + ", NY"
ahistoric_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208 West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,Aug-82,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,Jul-61,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,Feb-05,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,FF0D04EF-1D9A-47D6-8F20-8BDBDBCE2792,Sidewalk,Le Cafe Coffee,Le Cafe Coffee LLC,Le Cafe Coffee LLC,145,4th Avenue,Manhattan,10003,"145 4th Avenue, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [194]:
#9. Fix the SLA licence values which are corrupted
def replace_sla_with_default(value):
    if pd.notna(value) and isinstance(value, str):
        # Attempt to extract a 7-digit number from the string
        extracted_number = re.search(r'\b\d{7}\b', value)
        
        if extracted_number:
            return extracted_number.group()
        else:
            return '0000000'
    else:
        return value

# Apply the function to the 'Column' column
ahistoric_df['SLA Serial Number'] = ahistoric_df['SLA Serial Number'].apply(replace_sla_with_default)


# Print the updated DataFrame
ahistoric_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208 West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,0000000,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,0000000,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,0000000,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,FF0D04EF-1D9A-47D6-8F20-8BDBDBCE2792,Sidewalk,Le Cafe Coffee,Le Cafe Coffee LLC,Le Cafe Coffee LLC,145,4th Avenue,Manhattan,10003,"145 4th Avenue, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [195]:
ahistoric_df.to_csv('cleaned_historic_inspections.csv', index=False)